# Auth Code Grant Flow
Authorization Code Flow is one of the most common ones along with Hybrid Flow, in fact Hybrid Flow leverages the Auth Code Grant flow but extends it's functionality. 

In the Auth Code Grant, its purpose is to request an authorization code, known as the `auth_code` which the Client Application will then exchange securely using its own Apps credentials (`client_secret`) for an `access_token`.

> [!NOTE] 📝 Authorization Code Grant/Flow requires user interaction. The user must sign-in interactively and satisfy the MFA policy.

The flow execution looks something like this:
```sql
Client ----> [Authorization Endpoint]
                       \
                        User Authenticates
                       /
Client <---- [Authorization Code]
           |
           | (POST with code)
           v
[Token Endpoint] ----> [Tokens]
```

### Setup
Setup the following variables with your own application information, you can fine them under Application registrations.

Set the scope to the permissions you would like to grant to the application.

In [ ]:
import sys
sys.path.append('../')
import OAuth2_Flows
import pyperclip

## Required Variables Setup
Below we are setting up our variables.
- Note that the `redirect_uri` needs to match that on the App Registration

> Note that in the scope you could also request something like: `scope = 'openid email profile offline_access https://graph.microsoft.com/.default'`, and that would also return an id token along with a refresh token. This by the specification is no longer considered Auth Code flow, and it is using the Hybrid Flow leveraging the OpenID Connect (OIDC) spec, which allows for this type of behavior.

In [ ]:
tenant_id = ''
client_id = ''
redirect_uri = ''

scope = 'offline_access https://graph.microsoft.com/.default' #offline_access is required for refresh token
state = "A1B2C3D4E5F6"

The **`auth_code_flow`** function, returns the complete URL based on the parameters provided above to the authorization (`/authorize`) endpoint.
The user must then sign-in with there identity and grant the permission to the `scope` the application has requested.

> [!NOTE] 📝 The code will copy (so you don't have to 😉) the URL for you. You can either directly paste it in the browser or just click the hyperlink to open a new window. 

In [ ]:
complete_auth_url = OAuth2_Flows.auth_code_flow(tenant_id, client_id, redirect_uri, scope, state)
pyperclip.copy(complete_auth_url) # Copy to clipboard
print(f'Complete URL w/ params - Paste In Browser: \n{complete_auth_url}')

In the response, you should have received the Authorization Code as a `code` query string parameter within the URL. Copy the `code` and run below.
> [!TIP] 🔥 You can use the [**urlyzer**](https://github.com/ManuelBerrueta/urlyzer) tool to parse the url  for analysis and also to make it easier for you to copy the code 🙂    
> [!NOTE] 📝 You will also need the `client_secret` for this next step.

In [ ]:
auth_code = input('Enter the code from the URL: ')

In [ ]:
client_secret = input('Enter the client secret: ')

Now we can use the auth_code and client_secret to get the access token and refresh token.

In [ ]:
access_token, refresh_token, id_token = OAuth2_Flows.request_access_token(tenant_id, client_id, redirect_uri, auth_code, client_secret)
print(f'Access Token: {access_token}')
print(f'Refresh Token: {refresh_token}')
print(f'ID Token: {id_token}') # id_token is not returned in regular Auth Code Flow

---     

# Refresh Token Request
If the token expires, we can also request a new one using the **`refresh_token`** to make a request to the `/token` endpoint.
> [!NOTE] 📝 The `grant_type` in this case will be `refresh_token`

In [ ]:
refresh_token = input('Enter the refresh token: ')

In [ ]:
client_secret = input('Enter the client secret: ')

In [ ]:
access_token, refresh_token, id_token = OAuth2_Flows.refresh_token(tenant_id, client_id, refresh_token, client_secret)
print(f'Access Token: {access_token}')
print(f'Refresh Token: {refresh_token}')
print(f'ID Token: {id_token}') # id_token is not returned in regular Auth Code Flow

## Getting Refresh Token with a different scope
An additional, interesting fact is that you COULD request additional scopes with the refresh token.
> [!NOTE] 📝 The user must have consented to the additional scope to the application at some point or the application must have admin consent.

You can give it a try:

In [ ]:
scope = 'https://vault.azure.net/user_impersonation' #Change this to another scope you want to test

In [ ]:
refresh_token = input('Enter the refresh token: ')

In [ ]:
client_secret = input('Enter the client secret: ')

In [ ]:
tokens = OAuth2_Flows.refresh_token(tenant_id, client_id, refresh_token, client_secret, scope)
access_token = tokens[0]
refresh_token = tokens[1]
id_token = tokens[2] # id_token is not returned in regular Auth Code Flow
print(f'Access Token: {access_token}')
print(f'Refresh Token: {refresh_token}')
print(f'ID Token: {id_token}') # id_token is not returned in regular Auth Code Flow